In [1]:
import torch
from torch import nn
from torch.nn import functional as F

class Lenet5(nn.Module):
    """
    for cifar10 dataset
    """
    def __init__(self):
        super(Lenet5, self).__init__()

        self.conv_unit = nn.Sequential(
            #x:[batchsz, 3, 32, 32]==>[b, 6, 20, 20]
            nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0),
            nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
            #
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
            #

        )
        # flatten
        # fc_unit
        self.fc_unit = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

        # [b, 3, 32, 32]
        tmp = torch.randn(2, 3, 32, 32)
        out = self.conv_unit(tmp)
        # [b, 16, 5, 5]
        print("conv_unit: ",out.shape)

        #use Cross Entropy Loss,contain softmax
        # self.criteon = nn.CrossEntropyLoss()



    def forward(self, x):
        """

        :param x: [b, 3, 32, 32]
        :return:
        """
        batchsz = x.size(0)
        # [b, 3, 32, 32]=>[b, 16, 5, 5]
        x = self.conv_unit(x)
        # [b, 16, 5, 5]=>[b, 16*5*5]
        x = x.view(batchsz, 16*5*5)
        # x = x.view(batchsz, -1)
        # [b, 16*5*5] => [b, 10]
        logits = self.fc_unit(x)

        # [b, 10]
        # pred = F.softmax(logits, dim=1)
        # loss = self.criteon(logits, y)
        return  logits

def main():

    net = Lenet5()

    tmp = torch.randn(2, 3, 32, 32)
    out = net(tmp)
    # [b, 16, 5, 5]
    print("lenet_unit: ", out.shape)

if __name__ == '__main__':

    main()

conv_unit:  torch.Size([2, 16, 5, 5])
lenet_unit:  torch.Size([2, 10])


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch import nn
from torch import optim
# from dragon_dl.dragon_pytorch.lesson45.resnet import ResNet18

def main():
    batchsz = 128

    cifar_train = datasets.CIFAR10('data', True, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ]), download=True)
    cifar_train = DataLoader(cifar_train, batch_size=batchsz, shuffle=True)

    cifar_test = datasets.CIFAR10('data', False, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ]), download=True)
    cifar_test = DataLoader(cifar_test, batch_size=batchsz, shuffle=True)

    x, label = iter(cifar_train).next()
    print('x:', x.shape, 'label:', label.shape)

    device = torch.device('cuda')
    # model = Lenet5().to(device)
    model = ResNet18().to(device)
    # logits => Softmax => pred
    criteon = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr = 1e-3)
    print(model)

    for epoch in range(1000):

        model.train()
        for batchidx, (x, label) in enumerate(cifar_train):
            # [b, 3, 32, 32]
            # [b]
            x, label = x.to(device), label.to(device)

            logits = model(x)
            # logits: [b, 10]
            # label: [b] here doesn't need one-hot and probability
            # loss: tensor scalar
            loss = criteon(logits, label)

            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        #
        print(epoch, 'loss:', loss.item())

        model.eval()
        with torch.no_grad():
            # test
            total_correct = 0
            total_num = 0
            for x, label in cifar_test:
                # [b, 3, 32, 32]
                # [b]
                x, label = x.to(device), label.to(device)

                # [b, 10]
                logits = model(x)
                # [b]
                pred = logits.argmax(dim=1)
                # [b] vs [b] => scalar tensor
                total_correct += torch.eq(pred, label).float().sum().item()
                total_num += x.size(0)

            acc = total_correct / total_num
            print(epoch, acc)




if __name__ == '__main__':
    main()

Files already downloaded and verified
Files already downloaded and verified
x: torch.Size([128, 3, 32, 32]) label: torch.Size([128])
ResNet18(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blk1): ResBlk(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (extra): Sequential(
      (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (blk2): ResBlk(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=

 23%|██▎       | 39403520/170498071 [00:50<01:50, 1186025.94it/s]

0 loss: 1.0036799907684326
0 0.5901
1 loss: 0.7609409093856812
1 0.6118
2 loss: 0.7094250321388245
2 0.6885
3 loss: 0.47448891401290894
3 0.6217
4 loss: 0.4469822943210602
4 0.6546
5 loss: 0.23568415641784668
5 0.6957
6 loss: 0.47187119722366333
6 0.6813
7 loss: 0.23529155552387238
7 0.6801
8 loss: 0.16182735562324524
8 0.695
9 loss: 0.054022323340177536
9 0.6662
10 loss: 0.3160094618797302
10 0.6849
11 loss: 0.16224835813045502
11 0.693
12 loss: 0.07230500876903534
12 0.6763
13 loss: 0.13372336328029633
13 0.6494
14 loss: 0.25781184434890747
14 0.6845
15 loss: 0.04835467040538788
15 0.6927
16 loss: 0.0281381793320179
16 0.7055
17 loss: 0.10492197424173355
17 0.6841
18 loss: 0.05755515769124031
18 0.6859
19 loss: 0.12506401538848877
19 0.676
20 loss: 0.13523361086845398
20 0.684
21 loss: 0.021918747574090958
21 0.7001
22 loss: 0.05151817947626114
22 0.6933
23 loss: 0.07118099927902222
23 0.6761
24 loss: 0.03691953048110008
24 0.6989
25 loss: 0.07858820259571075
25 0.6933
26 loss: 0.055